# 환경 확인
현재 프로젝트의 경로를 확인하면서 프로젝트 경로로 디렉토리 이동을 수행합니다.

In [1]:
from chrisbase.io import environ_to_dataframe

environ_to_dataframe(max_value_len=200)

,key,value
0,CHROME_CONFIG_HOME,/home/chris/.config/chrome-remote-desktop/chrome-config
1,CHROME_REMOTE_DESKTOP_SESSION,1
2,CINNAMON_2D,1
3,CINNAMON_SLOWDOWN_FACTOR,0.0001
4,CINNAMON_SOFTWARE_RENDERING,1
5,CINNAMON_VERSION,5.2.7
6,CLICOLOR,1
7,CLICOLOR_FORCE,1
8,CLUTTER_DEFAULT_FPS,15
9,CLUTTER_IM_MODULE,xim


In [2]:
from chrisbase.util import to_dataframe
from chrislab.common.util import GpuProjectEnv

env = GpuProjectEnv(project_name="DeepKorean", working_gpus="0,1")
to_dataframe(env)

,key,value
0,hostname,chris137
1,hostaddr,129.254.164.137
2,python_path,/dat/anaconda3/envs/DeepKorean-23.03/bin/python
3,project_name,DeepKorean
4,project_path,/dat/proj/DeepKorean-23.03
5,working_path,/dat/proj/DeepKorean-23.03
6,running_file,tests/1-doc_cls-train.ipynb
7,working_gpus,"0,1"
8,number_of_gpus,4
9,cuda_home_dir,/usr/local/cuda-12.0


# 각종 설정
모델 하이퍼파라메터(hyperparameter)와 저장 위치 등 설정 정보를 선언합니다.

In [ ]:
from ratsnlp.nlpbook.classification import ClassificationTrainArguments

args = ClassificationTrainArguments(
    pretrained_model_name="pretrained/KcBERT-Base",
    downstream_corpus_name="nsmc",
    downstream_corpus_root_dir="data",
    downstream_model_dir="checkpoints/nsmc",
    batch_size=32,
    learning_rate=5e-5,
    max_seq_length=128,
    epochs=3,
    tpu_cores=0,
    seed=7,
)

# 랜덤 시드 고정
학습 재현을 위해 랜덤 시드를 고정합니다.

In [ ]:
from ratsnlp import nlpbook

nlpbook.set_seed(args)

# 로거 설정
메세지 출력 등을 위한 logger를 설정합니다.

In [ ]:
nlpbook.set_logger(args)

# 말뭉치 다운로드
실습에 사용할 말뭉치를 다운로드합니다.

In [ ]:
from Korpora import Korpora

Korpora.fetch(
    corpus_name=args.downstream_corpus_name,
    root_dir=args.downstream_corpus_root_dir,
)

# 토크나이저 준비
토큰화를 수행하는 토크나이저를 선언합니다

In [ ]:
from transformers import BertTokenizer

tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)
print(tokenizer.tokenize("안녕하세요. 반갑습니다."))
tokenizer

# 학습데이터 구축
학습데이터를 구축합니다.

In [ ]:
from ratsnlp.nlpbook.classification import NsmcCorpus, ClassificationDataset
from torch.utils.data import DataLoader, RandomSampler

corpus = NsmcCorpus()
train_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="train",
)
train_dataloader = DataLoader(
    train_dataset,
    batch_size=args.batch_size,
    sampler=RandomSampler(train_dataset, replacement=False),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

# 평가데이터 구축
학습 중에 사용할 평가데이터를 구축합니다.

In [ ]:
from torch.utils.data import SequentialSampler

val_dataset = ClassificationDataset(
    args=args,
    corpus=corpus,
    tokenizer=tokenizer,
    mode="test",
)
val_dataloader = DataLoader(
    val_dataset,
    batch_size=args.batch_size,
    sampler=SequentialSampler(val_dataset),
    collate_fn=nlpbook.data_collator,
    drop_last=False,
    num_workers=args.cpu_workers,
)

# 모델 초기화
사전학습 모델을 읽고, 문서 분류를 수행할 모델을 초기화합니다.

In [ ]:
from transformers import BertConfig, BertForSequenceClassification

pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=corpus.num_labels,
)
model = BertForSequenceClassification.from_pretrained(
    args.pretrained_model_name,
    config=pretrained_model_config,
)

# 학습 준비
Task와 Trainer를 준비합니다.

In [ ]:
from ratsnlp.nlpbook.classification import ClassificationTask

task = ClassificationTask(model, args)

In [ ]:
trainer = nlpbook.get_trainer(args)

# 학습 개시
준비한 데이터와 모델로 학습을 시작합니다. 학습 결과물은 미리 세팅한 위치(`args.downstream_model_dir`)에 저장됩니다.

In [ ]:
trainer.fit(
    task,
    train_dataloaders=train_dataloader,
    val_dataloaders=val_dataloader,
)